In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df.head()

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [5]:
data = df[['aqi']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled.shape

(20978, 1)

In [8]:
scaled = scaled.reshape((20978,))

In [9]:
scaled.shape

(20978,)

In [10]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [12]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [13]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [14]:
# batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [15]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0143 - root_mean_squared_error: 0.1162 - val_loss: 0.0063 - val_root_mean_squared_error: 0.0787 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0085 - root_mean_squared_error: 0.0919 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0765 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0082 - root_mean_squared_error: 0.0905 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0765 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0081 - root_mean_squared_error: 0.0898 - val_loss: 0.0058 - val_root_mean_squared_error: 0.0762 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0081 - root_mean_squared_error: 0.0900 - val_loss: 0.0058 - val_root_mean_squared_error: 0.0761 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0078 - root_mean_squared_err

In [16]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [17]:
poll = np.array(df["aqi"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 4.5698357


In [20]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 2.179250307381153


In [21]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 97.82074969261885 %


In [22]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 3.554779


In [23]:
forecast

array([[0.39407456, 0.37053648, 0.3643689 , ..., 0.34418464, 0.332134  ,
        0.34136724],
       [0.35893464, 0.3555043 , 0.3952383 , ..., 0.33646718, 0.34552473,
        0.33744895],
       [0.3568992 , 0.39295065, 0.40369314, ..., 0.34310958, 0.33632708,
        0.33412918],
       ...,
       [0.09757732, 0.11165283, 0.11279416, ..., 0.12293139, 0.11700513,
        0.12039022],
       [0.09663095, 0.10452829, 0.11039461, ..., 0.11661612, 0.12020627,
        0.12451528],
       [0.09534045, 0.10364802, 0.11180894, ..., 0.1194696 , 0.12369837,
        0.13182202]], dtype=float32)

In [24]:
testPredict

array([[175.10715, 173.67197, 173.29591, ..., 172.06522, 171.33046,
        171.89343],
       [172.96457, 172.7554 , 175.1781 , ..., 171.59467, 172.14693,
        171.65453],
       [172.84045, 175.03862, 175.69362, ..., 171.99966, 171.58612,
        171.4521 ],
       ...,
       [157.02884, 157.88707, 157.95667, ..., 158.57475, 158.21341,
        158.41982],
       [156.97115, 157.45267, 157.81035, ..., 158.1897 , 158.4086 ,
        158.67133],
       [156.89246, 157.39899, 157.89659, ..., 158.36368, 158.62152,
        159.11685]], dtype=float32)

In [25]:
y_test_true

array([[173.9278 , 173.5361 , 173.27498, ..., 168.83585, 169.22752,
        169.61922],
       [173.5361 , 173.27498, 172.8833 , ..., 169.22752, 169.61922,
        170.01091],
       [173.27498, 172.8833 , 174.45006, ..., 169.61922, 170.01091,
        169.22752],
       ...,
       [155.64897, 155.77954, 155.77954, ..., 157.99911, 158.78249,
        159.56587],
       [155.77954, 155.77954, 156.17123, ..., 158.78249, 159.56587,
        159.56587],
       [155.77954, 156.17123, 156.56291, ..., 159.56587, 159.56587,
        159.69643]], dtype=float32)

In [26]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [27]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 2.0341
RMSE for segment 2: 2.5859
RMSE for segment 3: 2.9526
RMSE for segment 4: 3.1901
RMSE for segment 5: 3.3531
RMSE for segment 6: 3.4528
RMSE for segment 7: 3.5380
RMSE for segment 8: 3.6082
RMSE for segment 9: 3.6480
RMSE for segment 10: 3.6954
RMSE for segment 11: 3.7318
RMSE for segment 12: 3.7530
RMSE for segment 13: 3.7717
RMSE for segment 14: 3.7876
RMSE for segment 15: 3.8016
RMSE for segment 16: 3.8144
RMSE for segment 17: 3.8169
RMSE for segment 18: 3.8212
RMSE for segment 19: 3.8405
RMSE for segment 20: 3.8424
RMSE for segment 21: 3.8329
RMSE for segment 22: 3.8395
RMSE for segment 23: 3.8379
RMSE for segment 24: 3.8707
RMSE for segment 25: 3.9250
RMSE for segment 26: 3.9980
RMSE for segment 27: 4.0602
RMSE for segment 28: 4.1127
RMSE for segment 29: 4.1473
RMSE for segment 30: 4.1648
RMSE for segment 31: 4.1971
RMSE for segment 32: 4.2324
RMSE for segment 33: 4.2420
RMSE for segment 34: 4.2747
RMSE for segment 35: 4.2913
RMSE for segment 36: 4.3030
R

In [28]:
model.save("aqi_1dcnn.keras")